In [ ]:
import numpy as np
from cube2x2 import Cube
from visualize import visualize_cube

In [ ]:
pol = np.loadtxt('Q_init.txt')
pol *= 0.05

In [ ]:
pol = np.where(pol == 0., -150, pol)

In [ ]:
pol.shape

In [ ]:
V = pol.copy()

In [ ]:
cube = Cube()

In [ ]:
def n_stepTD(episodes, n, policy, V, alpha):
    
    for ep in range(episodes):
        
        cube.shuffle_state()
        s_0 = cube.current_s
        s_list = [s_0]
        r_list = [0]
        a_list = []
        
        T = 1000 # early stop at 1000 ? takes ~2 min per ep
        t_step = 0
        s = s_0
        while True:
            
            A = np.argmax(policy[s])
            a_list.append(A)
            
            if t_step < T:
                cube.step(A)
                s_n = cube.current_s
                r = -1 # constant
                
                s_list.append(s_n)
                r_list.append(r)
            
                if s_n == cube.terminal_s:
                    T = t_step + 1
            
            tau = t_step - n + 1
            if tau >= 0:
                
                G = sum(r_list[tau+1:min(tau+n, T)])
                if tau + n < T:
                    G = G + V[s_list[tau+n],A]
                
                V[s_list[tau],A] = V[s_list[tau],A] + alpha * (G - V[s_list[tau],A])
                
            t_step += 1
            
            if tau == T - 1:
                if s_n != cube.terminal_s:
                    for i,s in enumerate(s_list):
                        V[s,a_list[i]] -= 1
                break
                
        print('Episode {} done'.format(ep+1))
    
    return V

In [ ]:
V_pol = n_stepTD(200, 6, pol, V, alpha=0.5)

In [ ]:
np.savetxt('nstep_Q.txt', V_pol)